# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:

#read in files
file_path = "Output\CityWeatherData.csv"

# Read the census data 
df = pd.read_csv(file_path)

df.head()


,City,Country,Latitude,Longitude,Date,Wind_Speed,Humidity,Max_Temp,Cloudiness
0,Arraial do Cabo,BR,-22.9661,-42.0278,1.646991e+09,19.57,88.0,75.06,0.0
1,Boa Vista,BR,2.8197,-60.6733,1.646990e+09,4.61,78.0,76.98,40.0
2,Torbay,CA,47.6666,-52.7314,1.646991e+09,20.71,89.0,29.75,40.0
3,Gornyy,RU,55.1153,83.9059,1.646991e+09,9.13,80.0,4.17,57.0
4,Chui,UY,-33.6971,-53.4616,1.646991e+09,7.34,73.0,58.10,69.0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        562 non-null    object 
 1   Country     559 non-null    object 
 2   Latitude    562 non-null    float64
 3   Longitude   562 non-null    float64
 4   Date        562 non-null    float64
 5   Wind_Speed  562 non-null    float64
 6   Humidity    562 non-null    float64
 7   Max_Temp    562 non-null    float64
 8   Cloudiness  562 non-null    float64
dtypes: float64(7), object(2)
memory usage: 39.6+ KB


In [22]:
dropna_weather_data = df.dropna()
dropna_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 0 to 561
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        559 non-null    object 
 1   Country     559 non-null    object 
 2   Latitude    559 non-null    float64
 3   Longitude   559 non-null    float64
 4   Date        559 non-null    float64
 5   Wind_Speed  559 non-null    float64
 6   Humidity    559 non-null    float64
 7   Max_Temp    559 non-null    float64
 8   Cloudiness  559 non-null    float64
dtypes: float64(7), object(2)
memory usage: 43.7+ KB


In [101]:
dropna_weather_data.Humidity.max()

100.0

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [92]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_weather_data[["Latitude", "Longitude"]]

humidity = dropna_weather_data["Humidity"]

In [106]:
# BONUS
# Plot as a Terrain map
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [78]:
mask = (dropna_weather_data.Max_Temp < 80) & (dropna_weather_data.Max_Temp > 70) & (dropna_weather_data.Wind_Speed < 10) & (dropna_weather_data.Cloudiness == 0)

perfect_vacation = dropna_weather_data.loc[mask]
perfect_vacation

,City,Country,Latitude,Longitude,Date,Wind_Speed,Humidity,Max_Temp,Cloudiness
126,Pedernales,DO,18.0384,-71.7440,1.646991e+09,2.68,67.0,76.53,0.0
224,Mengcheng,CN,33.2579,116.5655,1.646991e+09,9.78,42.0,76.60,0.0
354,Kahului,US,20.8947,-156.4700,1.646991e+09,5.01,68.0,75.85,0.0
396,Nānākuli,US,21.3906,-158.1547,1.646991e+09,4.61,73.0,75.70,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
hotel_names = []
ratings = []
addresses = []
cities = []
countries = []
lats = []
longs = []

for _, row in perfect_vacation.iterrows():
    print(row.City)

    # geocoordinates
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = "hotel"

    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()
    
    hotel = places_data["results"][0]["name"]
    address = places_data["results"][0]["vicinity"]
    print(places_data["results"][0]["name"])
    print(places_data["results"][0]["vicinity"])
    
    # put it all in a list
    hotel_names.append(hotel)
    addresses.append(address)
    cities.append(row.City)
    countries.append(row.Country)
    lats.append(row.Latitude)
    longs.append(row.Longitude)

Pedernales
Vista de Águilas Ecolodge
Calle 16 de Agosto No. 27, Pedernales
Mengcheng
Jinjiang Star pinshang Hotel
Building 19, Zone C, Zhuangzhou, 139 Liuhai Rd, Mengcheng County, Zhuangzhou
Kahului
Maui Seaside Hotel
100 W Kaahumanu Ave, Kahului
Nānākuli


IndexError: list index out of range

In [87]:
hotel_names

['Vista de Águilas Ecolodge',
 'Jinjiang Star pinshang Hotel',
 'Maui Seaside Hotel']

In [88]:
hotel_df = pd.DataFrame()
hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df['Latitude'] = lats
hotel_df['Longitude'] = longs
hotel_df['Hotel Name'] = hotel_names
hotel_df['Address'] = addresses

hotel_df

,City,Country,Latitude,Longitude,Hotel Name,Address
0,Pedernales,DO,18.0384,-71.7440,Vista de Águilas Ecolodge,"Calle 16 de Agosto No. 27, Pedernales"
1,Mengcheng,CN,33.2579,116.5655,Jinjiang Star pinshang Hotel,"Building 19, Zone C, Zhuangzhou, 139 Liuhai Rd..."
2,Kahului,US,20.8947,-156.4700,Maui Seaside Hotel,"100 W Kaahumanu Ave, Kahului"


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Address}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [107]:
# Add marker layer ontop of heat map
# create layers
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 max_intensity=100, point_radius=2)


fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))